In [43]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt   # main plotting library
import re
import preprocessor as p 
import os

import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from wordcloud import WordCloud   
import json
import urllib
from bs4 import BeautifulSoup



nltk.download('punkt')
nltk.download('stopwords')
sw = set(stopwords.words('english'))
sid = SentimentIntensityAnalyzer() 

def sentiment_analyzer(t):
    cleaned = word_tokenize(re.sub("[^a-zA-Z]", " ", p.clean(t)).strip().lower())
    token_list = [w for w in cleaned if not w in sw]  
    s=" ".join(token_list)
    return sid.polarity_scores(s)


directory = "./data/nytimes/"

articles = []

outfile = open("scores01.csv", "a+")
f = open(directory+"01.json", "r")
articles =json.load(f)['response']['docs']
f.close()

for article in articles:
    abstract = article['abstract'].lower()
    if "outbreak" in abstract or "virus" in abstract or "covid" in abstract:
        url = article['web_url']
        if "video" in url or "interactive" in url:
            print(url)
        else:
            c = urllib.request.urlopen(url)
            parsed = BeautifulSoup(c, 'html.parser')
            articleBody = parsed.find(attrs={"name": "articleBody"})
            if articleBody is not None:
                textbody = articleBody.text
                scores = sentiment_analyzer(textbody)
                toWrite = str(scores['neg'])+","+ str(scores['neu'])+","+str(scores['pos'])+","+str(scores['compound'])+","+article['pub_date']

                outfile.write(toWrite+"\n")
            else:
                print(url)

                                    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


https://www.nytimes.com/interactive/2020/01/21/world/asia/china-coronavirus-maps.html
https://www.nytimes.com/video/world/asia/100000006936419/coronavirus-china-wuhan.html
https://www.nytimes.com/interactive/2020/world/coronavirus-maps.html


In [44]:
outfile.close() 